<a href="https://colab.research.google.com/github/kelvinj1979/Data_Science/blob/main/Robot_Trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Configuración del ambiente

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from bs4 import BeautifulSoup
import requests
import time
import datetime
from IPython.display import clear_output
import plotly.express as px
import warnings

!python -V
print('------')
!pip show Pandas | grep 'Name\|Version'
print('------')
!pip show Numpy | grep 'Name\|Version'
print('------')
!pip show Matplotlib | grep 'Name\|Version'


global df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision

Python 3.10.12
------
Name: pandas
Version: 2.1.4
------
Name: numpy
Version: 1.26.4
        Name: lapack-lite
        Name: tempita
        Name: dragon4
        Name: libdivide
        Name: Meson
        Name: spin
        Name: OpenBLAS
        Name: LAPACK
        Name: GCC runtime library
        Version 3.1, 31 March 2009
                               Version 3, 29 June 2007
          5. Conveying Modified Source Versions.
          14. Revised Versions of this License.
        Name: libquadmath
------
Name: matplotlib
Version: 3.7.1


# 2. Obtención de datos

In [9]:
def importar_base_bitcoin():
    global df_bitcoin, precio_actual, tendencia, media_bitcoin
    # Obtenga los datos de los últimos 7 días en intervalos de 5 minutos
    fecha_actual = datetime.datetime.now()
    fecha_inicio = datetime.datetime.now() - datetime.timedelta(days = 7)
    df_bitcoin = yf.download("BTC-USD", start = fecha_inicio, end=fecha_actual, interval="5m")

In [10]:
importar_base_bitcoin()

[*********************100%***********************]  1 of 1 completed


In [11]:
def extraer_tendencias():
    global precio_actual, tendencia
    url = 'https://coinmarketcap.com/'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    # Extraer precio y cambio porcentual en 1 hora
    td = soup.find('td',style = 'text-align:end')
    precio_actual = float(td.div.span.getText().replace('$','').replace(',',''))
    td = soup.find('td',style="text-align:end").findNextSibling()

    if td.span.span['class'][0] == 'icon-Caret-up':
      tendencia = 'Alta'
    elif td.span.span['class'][0] == 'icon-Caret-down':
      tendencia = 'Baja'

In [12]:
extraer_tendencias()

# 3. Limpieza de datos

In [17]:
def limpieza_datos():
  global media_bitcoin, df_bitcoin
  outliers = []
  df_bitcoin.index = df_bitcoin.index.tz_convert('America/Caracas')
  df_bitcoin.rename_axis('Fecha',inplace=True)
  df_bitcoin_limpio = df_bitcoin.copy()
  df_bitcoin_limpio = df_bitcoin_limpio[~df_bitcoin_limpio.index.duplicated(keep='first')]
  df_bitcoin_limpio.reset_index(inplace=True)
  df_bitcoin_limpio['Fecha'] = df_bitcoin_limpio['Fecha'].dt.strftime("%d/%m/%Y %H:%M")
  df_bitcoin_limpio.set_index('Fecha',inplace=True)
  df_bitcoin_limpio.describe()

  '''plt.boxplot(df_bitcoin_limpio['Close'])
  plt.title('Datos sin limpiar')
  plt.grid(axis='y')
  plt.show()'''

  df_bitcoin_limpio['Close'].interpolate(method='linear',inplace=True)
  df_bitcoin_limpio = df_bitcoin_limpio[df_bitcoin_limpio['Volume'] > 0]

  for flier in plt.boxplot(df_bitcoin_limpio['Close'])['fliers']:
    outliers = flier.get_ydata()
    '''print(f'Outliers: {outliers}')'''

    for outlier in outliers:
      df_bitcoin_limpio = df_bitcoin_limpio[df_bitcoin_limpio['Close'] != float(outlier)]
  plt.close()

  Q1 = df_bitcoin_limpio['Close'].quantile(0.25)
  Q3 = df_bitcoin_limpio['Close'].quantile(0.75)

  '''print(f'Q1: {Q1}, Q3: {Q3}')'''

  df_bitcoin_limpio = df_bitcoin_limpio[(df_bitcoin_limpio['Close'] >= Q1) & (df_bitcoin_limpio['Close'] <= Q3)]

  '''plt.boxplot(df_bitcoin_limpio['Close'])
  plt.grid(axis='y')
  plt.title('Datos limpios')
  plt.show()'''

  media_bitcoin = df_bitcoin_limpio['Close'].mean()
  '''print(f'Media_bitcoin: {media_bitcoin}')'''

In [18]:
limpieza_datos()

# 4. Tomar decisiones

In [23]:
def tomar_decisiones():
    global algoritmo_decision, media_bitcoin, color_flecha
    media_bitcoin = df_bitcoin['Close'].mean()
    if precio_actual >= media_bitcoin and tendencia == 'baja':
      algoritmo_decision = 'Vender'
      color_flecha = 'red'
    elif precio_actual < media_bitcoin and tendencia == 'alta':
      algoritmo_decision = 'Comprar'
      color_flecha = 'green'
    else:
      algoritmo_decision = 'Esperar'
      color_flecha = 'grey'

# 5. Visualización

In [21]:
def visualizacion():
  global df_bitcoin, precio_actual, tendencia, media_bitcoin,  algoritmo_decision
  df_bitcoin['Promedio'] = media_bitcoin
  df_bitcoin.reset_index(inplace=True)
  plt = px.line(df_bitcoin, x='Fecha', y=(["Close",'Promedio']),hover_data={'Fecha': "|%d %B %Y %H:%M"},
                title='Precio historico del bitcoin (últimos 7 dias)',labels={'value':'Precio $ (BTC)'})

  plt.update_xaxes(tickformat="%d-%b-%Y",minor=dict(ticks='inside',showgrid=True),
                ticks= "inside",ticklen = 10,
                tick0 = datetime.datetime.today()-datetime.timedelta(days=7),
                rangeslider_visible=True,
                rangeselector=dict(
                buttons=list([
                dict(count=1, label="1d", step="day", stepmode="backward"),
                dict(count=2, label="2d", step="day", stepmode="backward"),
                dict(count=3, label="3d", step="day", stepmode="backward"),
                dict(count=4, label="4d", step="day", stepmode="backward"),
                dict(count=5, label="5d", step="day", stepmode="backward"),
                dict(step="all")])),
                tickformatstops = [
                dict(dtickrange=[None, 7200000], value="%H:%M m"),
                dict(dtickrange=[7200000, 43200000], value="%H:%M h"),
                dict(dtickrange=[43200000, 86400000], value="%d-%b-%Y"),])
  plt.add_annotation(x=df_bitcoin['Fecha'].iloc[-1], y=df_bitcoin['Close'].iloc[-1], text = algoritmo_decision, showarrow=True, arrowhead=1,arrowsize=1,arrowwidth=2,arrowcolor=color_flecha)
  #plt.annotate(algoritmo_decision, xy=(0.5, 0.9), xycoords='axes fraction', fontsize=20)
  plt.show()



# 6. Automatización

In [ ]:
while True:
    clear_output()
    importar_base_bitcoin()
    extraer_tendencias()
    limpieza_datos()
    tomar_decisiones()
    visualizacion()
    time.sleep(300)

[*********************100%***********************]  1 of 1 completed
